# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [42]:
#Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [43]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [44]:
#changing some columns data types
spaceship = spaceship.astype({"CryoSleep": bool, "VIP": bool})

#cabin column remapping
spaceship["Cabin"] = spaceship["Cabin"].apply(lambda x: x.split('/')[0] if isinstance(x, str) else x) 

#dropping passengerid & name column
spaceship = spaceship.drop(["PassengerId", "Name"], axis = 1)

#dropping null values and checking after
df = spaceship.dropna(how="any")
df.isna().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [45]:
#doing dummies for non-numerical
df = pd.get_dummies(df, columns=["HomePlanet", "Cabin", "Destination" ], dtype = int)
df

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,...,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0,...,1,0,0,0,0,0,0,0,0,1
1,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True,1,...,0,0,0,0,1,0,0,0,0,1
2,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0,...,0,0,0,0,0,0,0,0,0,1
3,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0,...,0,0,0,0,0,0,0,0,0,1
4,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True,1,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,0,...,0,0,0,0,0,0,0,1,0,0
8689,True,18.0,False,0.0,0.0,0.0,0.0,0.0,False,1,...,0,0,0,0,0,1,0,0,1,0
8690,False,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,1,...,0,0,0,0,0,1,0,0,0,1
8691,False,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,0,...,0,0,0,1,0,0,0,1,0,0


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [46]:
df["target"] = df["Transported"]

features = df.drop(columns=["Transported"])
target = df["Transported"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

**Perform Train Test Split**

In [47]:
normalizer = MinMaxScaler()
normalizer.fit(X_train)

MinMaxScaler()

In [48]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [49]:
knn = KNeighborsRegressor(n_neighbors = 10)
knn.fit(X_train_norm, y_train)

KNeighborsRegressor(n_neighbors=10)

In [50]:
tree = DecisionTreeRegressor(max_depth=10)
tree.fit(X_train_norm, y_train)

DecisionTreeRegressor(max_depth=10)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [51]:
bagging_reg = BaggingRegressor(KNeighborsRegressor(n_neighbors = 10),
                               n_estimators=100,
                               max_samples = 1000,
                               bootstrap=False)

In [52]:
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(bootstrap=False, estimator=KNeighborsRegressor(n_neighbors=10),
                 max_samples=1000, n_estimators=100)

In [53]:
pred = bagging_reg.predict(X_test_norm)

print("R2 score", bagging_reg.score(X_test_norm, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("MAE", mean_absolute_error(pred, y_test))

R2 score 0.957476231723266
RMSE 0.1031021571732314
MAE 0.03953056921995781


C:\Users\Amira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Random Forests

In [59]:
forest = RandomForestRegressor(n_estimators=100,
                             max_depth=20)

In [60]:
forest.fit(X_train_norm, y_train)

RandomForestRegressor(max_depth=20)

In [61]:
pred = forest.predict(X_test_norm)

print("R2 score", forest.score(X_test_norm, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("MAE", mean_absolute_error(pred, y_test))

R2 score 1.0
RMSE 0.0
MAE 0.0


C:\Users\Amira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Gradient Boosting

In [63]:
gb_reg = GradientBoostingRegressor(max_depth=20,
                                   n_estimators=100)

In [64]:
gb_reg.fit(X_train_norm, y_train)

GradientBoostingRegressor(max_depth=20)

In [66]:
pred = gb_reg.predict(X_test_norm)

print("R2 score", gb_reg.score(X_test_norm, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("MAE", mean_absolute_error(pred, y_test))

R2 score 0.9999999992944834
RMSE 1.328022703183049e-05
MAE 1.328001686974419e-05


C:\Users\Amira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


- Adaptive Boosting

In [67]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),
                            n_estimators=100)

In [68]:
ada_reg.fit(X_train_norm, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                  n_estimators=100)

In [69]:
pred = ada_reg.predict(X_test_norm)

print("R2 score", ada_reg.score(X_test_norm, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("MAE", mean_absolute_error(pred, y_test))

R2 score 1.0
RMSE 0.0
MAE 0.0


C:\Users\Amira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Which model is the best and why?

In [58]:
#comment here
# Random Forest and Adaptive boosting is the best model. I think in the KNN regression this two model works best.